# Создание различных ML моделей для определения спам сообщений

## Импорт необходимых библиотек

> Ячейки с установкой библиотек были приведены к типу `Raw`, чтобы они не выполнялись при автоматическом запуске всех ячеек.

In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pickle

In [2]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import nltk
from nltk.corpus import stopwords

In [3]:
import sys
sys.path.append('../utils')  # Добавление папки ../utils в список путей поиска модулей.
from preprocessing import count_tags, count_links, count_emojis, count_whitespaces, count_newlines, replace_tags, replace_links, remove_emojis, preprocess_text

## Чтение обработанного датасета

In [4]:
df = pd.read_csv('../data/preprocessed.csv', index_col=0)

In [5]:
df.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed
0,Добрый день! Отличается ли перечень необходимы...,0,0,0,0,0,0,добрый день отличается ли перечень необходимых...
1,Узбекистан. Рассматриваются обе формы,0,0,0,0,0,0,узбекистан рассматриваются обе формы
2,"Здравствуйте, а как проходит поступление после...",0,0,0,0,0,0,здравствуйте а как проходит поступление после ...
3,Спасибо большое за ответ!,0,0,0,0,0,0,спасибо большое за ответ
4,"Здравствуйте, а когда будет день открытых двер...",0,0,0,0,0,0,здравствуйте а когда будет день открытых двере...


## Подготовка выборок

In [6]:
texts = list(df['text_preprocessed'])  # Тексты
labels = list(df['label'])  # Метки (целевые значения)

In [7]:
additional_features = list(df[['emojis', 'whitespaces', 'links', 'tags']].values)  # Дополнительные числовые признаки

### Разделение на обучающую и тестовую выборки

In [8]:
X_train_text, X_test_text, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

In [9]:
X_train_num, X_test_num = train_test_split(
    additional_features, test_size=0.2, random_state=42, stratify=labels
)

### Подготовка стоп-слов

Загрузка английских и русских стоп-слов (`nltk`)

In [10]:
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))
english_stopwords = set(ENGLISH_STOP_WORDS)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\overklassniy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Объединение русских и английских стоп-слов

In [11]:
combined_stopwords = list(russian_stopwords.union(english_stopwords))

In [12]:
combined_stopwords

['than',
 'though',
 'много',
 'какой',
 'nowhere',
 'neither',
 'its',
 'but',
 'whose',
 'ему',
 'мой',
 'be',
 'throughout',
 'this',
 'кто',
 'сейчас',
 'another',
 'mine',
 'быть',
 'us',
 'чтобы',
 'each',
 'ourselves',
 'enough',
 'between',
 'has',
 'hereupon',
 'else',
 'уж',
 'during',
 'об',
 'only',
 'ведь',
 'he',
 'его',
 'further',
 'вот',
 'find',
 'against',
 'конечно',
 'seeming',
 'a',
 'по',
 'они',
 'она',
 'или',
 'том',
 'front',
 'such',
 'мне',
 'чем',
 'before',
 'amount',
 'где',
 'зачем',
 'via',
 'ни',
 'anyway',
 'how',
 'после',
 'one',
 'could',
 'hers',
 'in',
 'nothing',
 'ничего',
 'somewhere',
 'no',
 'перед',
 'yourselves',
 'and',
 'will',
 'which',
 'во',
 'do',
 'больше',
 'about',
 'thru',
 'thereafter',
 'wherever',
 'are',
 'hence',
 'with',
 'third',
 'whereas',
 'afterwards',
 'across',
 'many',
 'whither',
 'up',
 'thick',
 'whether',
 'anyone',
 'при',
 'twenty',
 'sometimes',
 'meanwhile',
 'три',
 'там',
 'whatever',
 'never',
 'was',
 '

Проверка на наличие слов `tag` и `link` в списке стоп-слов.

In [13]:
'TAG' in combined_stopwords, 'tag' in combined_stopwords, 'LINK' in combined_stopwords, 'link' in combined_stopwords

(False, False, False, False)

In [14]:
len(combined_stopwords)

469

### Подготовка текстовых и числовых признаков

#### Преобразование текста в числовой формат (TF-IDF)

In [15]:
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words=combined_stopwords,
    min_df=2,
    sublinear_tf=True
)

In [16]:
text_features_train = vectorizer.fit_transform(X_train_text)
text_features_test = vectorizer.transform(X_test_text)

#### Масштабирование числовых признаков

In [17]:
numerical_features_train = pd.DataFrame(X_train_num, columns=['emojis', 'whitespaces', 'links', 'tags'])
numerical_features_test = pd.DataFrame(X_test_num, columns=['emojis', 'whitespaces', 'links', 'tags'])

scaler = StandardScaler()
numerical_features_train_scaled = scaler.fit_transform(numerical_features_train)
numerical_features_test_scaled = scaler.transform(numerical_features_test)

#### Объединение текстовых и числовых признаков

In [18]:
X_train = hstack([text_features_train, numerical_features_train_scaled])
X_test = hstack([text_features_test, numerical_features_test_scaled])

#### Дополнительная синтетическая генерация

In [19]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

## Обучение моделей

### Logistic Regression

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE 

In [20]:
# 4. Обучение модели Logistic Regression
model_lr = LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)
model_lr.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

### Logistic Regression + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка с SMOTE

In [21]:
# 4. Обучение модели Logistic Regression
model_lr_sm = LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)
model_lr_sm.fit(X_train_resampled, y_train_resampled)

LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

### Random Forest

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

In [22]:
model_rf = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
model_rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

### Random Forest + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка с SMOTE

In [23]:
model_rf_sm = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
model_rf_sm.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(class_weight='balanced', random_state=42)

### Naive Bayes

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

Объединение текстов и числовых признаков без масштабирования

In [24]:
X_train_nc = hstack([text_features_train, X_train_num])
X_test_nc = hstack([text_features_test, X_test_num])

Обучение Naive Bayes

In [25]:
model_nb = MultinomialNB()
model_nb.fit(X_train_nc, y_train)

MultinomialNB()

### Naive Bayes + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

Объединение текстов и числовых признаков без масштабирования

In [26]:
X_train_nc = hstack([text_features_train, X_train_num])
X_test_nc = hstack([text_features_test, X_test_num])

In [27]:
smote = SMOTE(random_state=42)
X_train_resampled_nc, y_train_resampled_nc = smote.fit_resample(X_train_nc, y_train)

In [28]:
model_nb_sm = MultinomialNB()
model_nb_sm.fit(X_train_resampled_nc, y_train_resampled_nc)

MultinomialNB()

## Оценка качества моделей

Примеры тестовых сообщений

In [29]:
test_messages = [
    "Это честное сообщение от пользователя.",
    "🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎",
    "Зарабатывай миллионы **онлайн** прямо сейчас!",
    "Работа на дому, легкий доход. Пиши в личку!",
    "Привет! Как дела? У меня всё отлично.",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM\n\nhttps://t.me/ancleroyofficial",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or click the link on my bio",
    "steam gift 50$ - steamcommunity.com/gift-card/pay/50\n@everyone",
    "Давайте **вместе** будем писать про казино в чатах!!! Присоединяйтесь!",
    "Как же надоели эти сообщения про казино",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: stankin.ru",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru",
    "Поступление – это почти что казино! Лотерея!",
    "3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1",
    "Выиграл 345к дало x3450\n\nИграл тут: @jet_casino_ibot"
]

Функция для предсказания на новых данных

> Эта же функция находится в `../utils/preprocessing.py`.

In [30]:
def predict_message(text: str, vectorizer, scaler, model) -> tuple[int, list[float]]:
    """
    Предсказывает метку и вероятности для текста с использованием модели машинного обучения.

    Параметры:
        text: (str) - входной текст для анализа
        vectorizer: - векторизатор для преобразования текста
        scaler: - масштабатор числовых признаков
        model: - модель машинного обучения

    Возвращает:
        tuple[int, list[float]] - метка предсказания и вероятности
    """
    # Предварительная обработка текста
    text_preprocessed = preprocess_text(text)

    # Векторизация текста
    text_vector = vectorizer.transform([text_preprocessed])

    # Создание числовых признаков
    numerical_features = pd.DataFrame(
        [[count_emojis(text), count_whitespaces(text), count_links(text), count_tags(text)]],
        columns=['emojis', 'whitespaces', 'links', 'tags']
    )
    # Масштабирование числовых признаков
    numerical_features_scaled = scaler.transform(numerical_features)

    # Объединение текстовых и числовых признаков
    features = hstack([text_vector, numerical_features_scaled])

    # Предсказание
    prediction = model.predict(features)
    probabilities = model.predict_proba(features)
    return int(prediction[0]), *probabilities.tolist()

### Logistic Regression

In [31]:
y_pred = model_lr.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.99      0.94      4948
           1       0.99      0.92      0.96      6931

    accuracy                           0.95     11879
   macro avg       0.95      0.96      0.95     11879
weighted avg       0.95      0.95      0.95     11879

Confusion Matrix:
 [[4894   54]
 [ 544 6387]]


In [32]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_lr)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9500623, 0.0499377]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.1057015, 0.8942985]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.6340702, 0.3659298]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.0382523, 0.9617477]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.8523795, 0.1476205]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.2

### Logistic Regression + SMOTE

In [33]:
y_pred = model_lr_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.99      0.94      4948
           1       0.99      0.92      0.96      6931

    accuracy                           0.95     11879
   macro avg       0.95      0.96      0.95     11879
weighted avg       0.95      0.95      0.95     11879

Confusion Matrix:
 [[4887   61]
 [ 531 6400]]


In [34]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_lr_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9508327, 0.0491673]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.0904694, 0.9095306]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.6463019, 0.3536981]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.0310999, 0.9689001]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.8640727, 0.1359273]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.1

### Random Forest

In [35]:
y_pred = model_rf.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95      4948
           1       0.99      0.94      0.96      6931

    accuracy                           0.96     11879
   macro avg       0.95      0.96      0.95     11879
weighted avg       0.96      0.96      0.96     11879

Confusion Matrix:
 [[4859   89]
 [ 444 6487]]


In [36]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_rf)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.99, 0.01]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.3693672, 0.6306328]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.93, 0.07]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.04, 0.96]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.8805238, 0.1194762]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.1333667, 0.8666333]

Сообщение:

### Random Forest + SMOTE

In [37]:
y_pred = model_rf_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95      4948
           1       0.99      0.94      0.96      6931

    accuracy                           0.95     11879
   macro avg       0.95      0.96      0.95     11879
weighted avg       0.96      0.95      0.95     11879

Confusion Matrix:
 [[4857   91]
 [ 449 6482]]





In [38]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_rf_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.96, 0.04]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.3153031, 0.6846969]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.9590821, 0.0409179]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.03, 0.97]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9271416, 0.0728584]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.1591158, 0.8408842]



### Naive Bayes

In [39]:
y_pred = model_nb.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.53      1.00      0.69      4948
           1       1.00      0.36      0.53      6931

    accuracy                           0.63     11879
   macro avg       0.76      0.68      0.61     11879
weighted avg       0.80      0.63      0.60     11879

Confusion Matrix:
 [[4945    3]
 [4430 2501]]





In [40]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_nb)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9987193, 0.0012807]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 0
Вероятности: [0.9883945, 0.0116055]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.9945113, 0.0054887]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.9437713, 0.0562287]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9984714, 0.0015286]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 0
Вероятности: [0.9

### Naive Bayes + SMOTE

In [41]:
y_pred = model_nb_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.54      1.00      0.70      4948
           1       1.00      0.39      0.56      6931

    accuracy                           0.64     11879
   macro avg       0.77      0.69      0.63     11879
weighted avg       0.81      0.64      0.62     11879

Confusion Matrix:
 [[4945    3]
 [4227 2704]]





In [42]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_nb_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9988148, 0.0011852]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 0
Вероятности: [0.9908085, 0.0091915]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.9963198, 0.0036802]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.9372009, 0.0627991]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9989862, 0.0010138]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 0
Вероятности: [0.9

## Сохранение результатов

In [44]:
with open('../models/vectorizer.pkl', 'wb') as vec_file:
    pickle.dump(vectorizer, vec_file)

with open('../models/scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

print("Векторизатор и масштабатор сохранены!")

Векторизатор и масштабатор сохранены!


In [45]:
with open('../models/lr_model.pkl', 'wb') as lr_model_file:
    pickle.dump(model_lr, lr_model_file)

print("Модель логической регрессии без семплирования сохранена!")

Модель логической регрессии без семплирования сохранена!


In [46]:
with open('../models/lr_sm_model.pkl', 'wb') as lr_sm_model_file:
    pickle.dump(model_lr_sm, lr_sm_model_file)

print("Модель логической регрессии со семплированием сохранена!")

Модель логической регрессии со семплированием сохранена!


In [47]:
with open('../models/rf_model.pkl', 'wb') as rf_model_file:
    pickle.dump(model_rf, rf_model_file)

print("Модель случайного леса без семплирования сохранена!")

Модель случайного леса без семплирования сохранена!


In [48]:
with open('../models/rf_sm_model.pkl', 'wb') as rf_sm_model_file:
    pickle.dump(model_rf_sm, rf_sm_model_file)

print("Модель случайного леса со семплированием сохранена!")

Модель случайного леса со семплированием сохранена!


In [49]:
with open('../models/nb_model.pkl', 'wb') as nb_model_file:
    pickle.dump(model_nb, nb_model_file)

print("Модель Naive Bayes без семплирования сохранена!")

Модель Naive Bayes без семплирования сохранена!


In [50]:
with open('../models/nb_sm_model.pkl', 'wb') as nb_sm_model_file:
    pickle.dump(model_nb_sm, nb_sm_model_file)

print("Модель Naive Bayes со семплированием сохранена!")

Модель Naive Bayes со семплированием сохранена!
